## 그래프 2차 자동화

In [ ]:
!pip install pymysql
!pip install sqlalchemy
!pip install mysqlclient
!pip install pyecharts
!pip install chart_studio

     |████████████████████████████████| 43 kB 1.1 MB/s 
     |████████████████████████████████| 87 kB 3.2 MB/s 
  Created wheel for mysqlclient: filename=mysqlclient-2.1.0-cp37-cp37m-linux_x86_64.whl size=99973 sha256=670afd0062e4442b32cf392d1cfedc838debc57ddb38a950927a4f12e69cf344
  Stored in directory: /root/.cache/pip/wheels/97/d4/df/08cd6e1fa4a8691b268ab254bd0fa589827ab5b65638c010b4
Successfully built mysqlclient
     |████████████████████████████████| 135 kB 5.4 MB/s 
     |████████████████████████████████| 130 kB 36.3 MB/s 
     |████████████████████████████████| 64 kB 2.1 MB/s 


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import time
import datetime
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import numpy as np
import pymysql
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import font_manager, rc

import itertools
import operator

import base64
from io import BytesIO

from datetime import datetime, timedelta

from pyecharts.charts import Bar
from pyecharts.charts import Line
import plotly.offline as plyo
import cufflinks
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
# db = pymysql.connect(host=,db='upbit', charset='utf8')
# cursor = db.cursor(pymysql.cursors.DictCursor)
# sql = 'SELECT * FROM cj'
# cursor.execute(sql)
# result = cursor.fetchall()
# DF = pd.DataFrame(result)


In [ ]:
# date = ['2021-11-22','2021-11-23','2021-11-24','2021-11-25','2021-11-26','2021-11-27','2021-11-28','2021-11-29','2021-11-30']
# # lang = ['kor','eng']
# crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
# s = [0,1,2]

In [ ]:
# df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
# for i in date:
#   for j in crpyto_name:
#     df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)
# df

- DF 원본 from cj
- df 새로운 데이터 프레임

In [ ]:
def update_graphs():

    db = pymysql.connect(host=, charset='utf8')
    cursor = db.cursor(pymysql.cursors.DictCursor)
    sql = 'SELECT * FROM cj'
    cursor.execute(sql)
    result = cursor.fetchall()
    DF = pd.DataFrame(result)


    date = ['2021-11-22','2021-11-23','2021-11-24','2021-11-25','2021-11-26','2021-11-27','2021-11-28','2021-11-29','2021-11-30']
    # lang = ['kor','eng']
    crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']
    s = [0,1,2]

    df = pd.DataFrame(columns = ['date', 'crypto_name', 'positive_E','neutrual_E','negative_E','positive_K','neutrual_K','negative_K'])
    for i in date:
      for j in crpyto_name:
        df = df.append(pd.DataFrame([[i, j]], columns = ['date','crypto_name']), ignore_index=True)

    for k in crpyto_name:

      for j in date:

        for i in range(len(DF)):


          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
            if (DF['s'][i] == 0):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
              df['positive_K'][a] = DF['s.1'][i]

          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
            if (DF['s'][i] == 0):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
              df['positive_E'][a] = DF['s.1'][i]

          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
            if (DF['s'][i] == 1):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
              df['neutrual_K'][a] = DF['s.1'][i]

          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
            if (DF['s'][i] == 1):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
              df['neutrual_E'][a] = DF['s.1'][i]

          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'kor'):
            if (DF['s'][i] == 2):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'kor')].tolist()
              df['negative_K'][a] = DF['s.1'][i]

          if (DF['date'][i] == j) and (DF['coin_name'][i] == k) and (DF['lang'][i] == 'eng'):
            if (DF['s'][i] == 2):
              a = df.index[(df['date'] == j)&(df['crypto_name'] == k)&(DF['lang'][i] == 'eng')].tolist()
              df['negative_E'][a] = DF['s.1'][i]   


    df['positive_ratio_E'] = df['positive_E']/(df['positive_E']+df['neutrual_E']+df['negative_E'])
    df['positive_ratio_K'] = df['positive_K']/(df['positive_K']+df['neutrual_K']+df['negative_K'])


    df = df.fillna(0)
    daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)
    daily_conn = daily_engine.connect()
    df.to_sql(name='sentiment_result_graphing',con=daily_conn,if_exists='replace')
    print('업데이트 완료 : sentiment_result_graphing' , '\n' )

In [ ]:
update_graphs()

SyntaxError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 연습
DF = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/datasets/cryptocurrency_sentiment/sentiment_result_graphing.csv')

In [ ]:
def combine_senti_price():

    crypto_name_p = ['ADA','BTC','BCH','DOGE','EOS','ETH','OMG','SNT','XRP']

    for i in crypto_name_p:
        db = pymysql.connect(host=, charset='utf8')
        cursor = db.cursor(pymysql.cursors.DictCursor)
        sql = 'SELECT * FROM {0}'.format(i)
        cursor.execute(sql)
        result = cursor.fetchall()
        globals()[f'{i}'] = pd.DataFrame(result)



    crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]

    for j1,j2,k in zip(crypto_p,crypto_p,crypto_name_p):

      X = []

      for i in range(len(j1)):
        date_time_obj = datetime.strftime(j2['index'][i], '%Y-%m-%d')
        a = date_time_obj
        b = j2['close'][i]
        # c = j2['value'][i]
        d = j2['volume'][i]
        e = 0
        f = 0
        g = 0
        h = 0
        i = 0
        j = 0
        k = 0
        l = 0

        x = [a,b,d,e,f,g,h,i,j]
        X.append(x)
      globals()['{0}'.format(k)] = pd.DataFrame(X)
      globals()['{0}'.format(k)].columns = ['date','close_KRW','volume','positive_E','neutrual_E','negative_E',	'positive_K',	'neutrual_K',	'negative_K','positive_ratio_E','positive_ratio_K']

      crypto_p = [ADA,BTC,BCH,DOGE,EOS,ETH,OMG,SNT,XRP]
      crpyto_name = ['ada','bch','btc','doge','eos','eth','omg','snt','xrp']

      for k,l in zip(crypto_p,crpyto_name):

        for j in date:

            a = DF.index[(DF['date'] == j)&(DF['crypto_name'] == l)].tolist()
            b = k.index[(k['date'] == j)].tolist()
            k['positive_E'][b] = DF['positive_E'][a]
            k['neutrual_E'][b] = DF['neutrual_E'][a]
            k['negative_E'][b] = DF['negative_E'][a]
            k['positive_K'][b] = DF['positive_K'][a]
            k['neutrual_K'][b] = DF['neutrual_K'][a]
            k['negative_K'][b] = DF['negative_K'][a]
            k['positive_ratio_E'][b] = DF['positive_ratio_E'][a]
            k['positive_ratio_K'][b] = DF['positive_ratio_K'][a]

      daily_engine = create_engine("mysql+mysqldb:", encoding='utf-8',pool_size = 100000,max_overflow = 0)      
      daily_conn = daily_engine.connect()
      k.to_sql(name='graph_{0}'.format(j),con=daily_conn,if_exists='replace')
      print(' 업데이트 완료:', 'graph_{0}'.format(j), '\n' )
      

In [ ]:
combine_senti_price()

### 그래프 불러와서 만들기

In [ ]:
def graphing(coin, lang):
  ADA2 = ADA.replace(0,'')
  

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['close_KRW'],name='KRW',text=ADA2['close_KRW'],yaxis='y'))
fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['positive_E'],name='positive_E',text =ADA2['positive_E'] ,yaxis='y2'))
# fig.add_trace(go.Scatter(x=ADA2['date'],y=ADA2['negative_E'],name='negative_E',yaxis='y3'))

fig.update_traces(
    hoverinfo="name+x+text",
    line={"width": 0.5},
    marker={"size": 0.6},
    mode="lines+markers",
    showlegend=False
)


fig.update_layout(
    xaxis=dict(
        autorange=True,
        range=["2021-10-31", "2021-11-25"],
        rangeslider=dict(
            autorange=True,
            range=["2021-10-31", "2021-11-25"]
        ),
        type="date"
    ),
    yaxis=dict(
        anchor="x",
        autorange=True,
        domain=[0.3, 1],
        linecolor="#673ab7",
        mirror=True,
        range=[-60.0858369099, 1000],
        showline=True,
        side="left",
        tickfont={"color": "#673ab7"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#673ab7"},
        type="linear",
        zeroline=False
    ),
    yaxis2=dict(
        anchor="x",
        autorange=True,
        domain=[0, 0.1],
        linecolor="#E91E63",
        mirror=True,
        range=[29.3787777032, 100000],
        showline=True,
        side="right",
        tickfont={"color": "#E91E63"},
        tickmode="auto",
        ticks="",
        titlefont={"color": "#E91E63"},
        type="linear",
        zeroline=False
    )
)

fig.update_layout(
    dragmode="zoom",
    hovermode="x",
    legend=dict(traceorder="reversed"),
    height=600,
    template="plotly_white",
    margin=dict(
        t=100,
        b=100
    ),
)
